In [2]:
!pip install faiss-cpu langchain langchain-community langchain-core pypdf sentence-transformers transformers==4.52.4 torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 121.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.2/111.2 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 496.3/496.3 kB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.4/566.4 kB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.2/319.2 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import CharacterTextSplitter # Changed import
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [4]:
model_name = "mistralai/Mistral-Nemo-Instruct-2407"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/622 [00:00<?, ?B/s]

...

model-00002-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/4.91G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [5]:
def generate_text(prompt, max_length=512, num_return_sequences=1):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        num_return_sequences=num_return_sequences,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.7,
    )
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

In [7]:
pdf_path = "/content/TeleConnect_Customer_Service_Knowledge_Base.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(documents)

embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)
vectordb = FAISS.from_documents(chunks, embedding)

/tmp/ipython-input-2713714650.py:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name=embedding_model_name)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
from langchain_community.document_loaders import PyPDFLoader, CSVLoader
import os
from langchain_text_splitters import CharacterTextSplitter # Make sure this is imported if not already
from langchain_community.vectorstores import FAISS # Make sure this is imported if not already
from langchain_community.embeddings import HuggingFaceEmbeddings # Make sure this is imported if not already

# 1. Define your files (you can add as many as you want here)
# If using Colab, ensure these files are uploaded to the folder on the left
files_to_load = [
    "/content/TeleConnect_Customer_Service_Knowledge_Base.pdf",
    "/content/TeleConnect_Service_Plans.csv"
]

all_documents = []

# 2. Loop through files and load based on type
for file_path in files_to_load:
    if not os.path.exists(file_path):
        print(f"Skipping: {file_path} (File not found)")
        continue

    print(f"Loading: {file_path}...")

    if file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
        all_documents.extend(loader.load())

    elif file_path.endswith(".csv"):
        # CSV files are great for Google Sheets data
        loader = CSVLoader(file_path=file_path, encoding="utf-8")
        all_documents.extend(loader.load())

# 3. Split all documents into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
chunks = text_splitter.split_documents(all_documents)

# 4. Update the Vector Database (FAISS)
# This uses the same embedding logic from your original code
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
# Explicitly set device to 'cpu' for embeddings to save GPU memory
embedding = HuggingFaceEmbeddings(model_name=embedding_model_name, model_kwargs={'device': 'cpu'})
vectordb = FAISS.from_documents(chunks, embedding)

print(f"\nSuccess! Total chunks in knowledge base: {len(chunks)}")

Loading: /content/TeleConnect_Customer_Service_Knowledge_Base.pdf...
Loading: /content/TeleConnect_Service_Plans.csv...

Success! Total chunks in knowledge base: 4


In [10]:
def ask_question(query):
    docs = vectordb.similarity_search(query, k=1)
    context = "\n\n".join([doc.page_content for doc in docs])

    prompt = f"""You are a helpful assistant. Use the following context to answer the question.

Context:
{context}

Question: {query}
Answer:"""

    # Further reduced max_length to avoid OutOfMemoryError during generation
    result = generate_text(prompt, max_length=256)[0]
    return result.strip()

# Using the previous user query to get an answer
user_query = "how much the price of basic"
answer = ask_question(user_query)
print("\nAnswer:", answer.split("Answer:")[-1], "\n")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Answer:  The monthly price of the Basic plan is $25. 



# Task
Create an interactive chatbot GUI using `ipywidgets` and `IPython.display`. The chatbot will feature an input text area, an output display area, and a button. The logic will handle user input, call the `ask_question` function (which was previously defined), and update the output display with the chatbot's response. Finally, display the GUI.

## Import GUI Libraries

### Subtask:
Import `ipywidgets` and `IPython.display` to create and display the interactive chatbot interface.


**Reasoning**:
To create the interactive chatbot interface, I need to import the specified GUI libraries, `ipywidgets` and `IPython.display`.



In [17]:
import ipywidgets as widgets
from IPython.display import display

## Define Chatbot GUI Elements

### Subtask:
Create an input text area, an output display area, and a button using `ipywidgets`. These will form the interactive components of the chatbot.


**Reasoning**:
I need to create the `ipywidgets` elements as specified in the instructions: an input text area, an output display area, and a button. I will define `text_input`, `output_area`, and `ask_button` with their respective properties.



In [18]:
text_input = widgets.Textarea(
    value='',
    placeholder='Type your question here...',
    description='Your Question:',
    disabled=False,
    layout=widgets.Layout(width='700px', height='auto')
)

output_area = widgets.Output()

ask_button = widgets.Button(
    description='Ask',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click to ask the question',
    icon='comment'
)

print("Chatbot GUI elements created successfully.")

Chatbot GUI elements created successfully.


## Implement Chatbot Logic for GUI

### Subtask:
Write a function that handles user input from the GUI, calls the `ask_question` function, and updates the output display with the chatbot's response. This function will be linked to the button's action.


**Reasoning**:
I need to define the `on_button_click` function as described, which will handle the chatbot's core logic by taking user input, calling `ask_question`, updating the display, and clearing the input field. After defining the function, I will attach it to the `ask_button`'s `on_click` event.



In [19]:
def on_button_click(button):
    with output_area:
        output_area.clear_output()
        user_question = text_input.value
        print(f"You asked: {user_question}")

        if user_question:
            try:
                # Calling the ask_question function
                answer = ask_question(user_question)
                print(f"Chatbot: {answer}")
            except Exception as e:
                print(f"An error occurred: {e}")
        else:
            print("Please type a question.")

        # Clear the input text area after processing
        text_input.value = ''

# Attach the function to the button's on_click event
ask_button.on_click(on_button_click)

print("Chatbot logic and button event handler defined successfully.")

Chatbot logic and button event handler defined successfully.


**Reasoning**:
Now that the GUI elements are defined and the logic for the button click is implemented, the final step for this subtask is to display the interactive chatbot interface using `IPython.display.display`.



In [20]:
display(widgets.VBox([text_input, ask_button, output_area]))
print("Chatbot GUI displayed successfully.")

Chatbot GUI displayed successfully.


## Final task

### Subtask:
Confirm the successful creation and display of the GUI chatbot, ready for interaction.


## Summary:

### Q&A
The interactive chatbot GUI has been successfully created and displayed, ready for user interaction.

### Data Analysis Key Findings
*   Required libraries, `ipywidgets` and `IPython.display`, were successfully imported without errors, making them available for GUI construction.
*   The essential GUI components were created: a `Textarea` widget for user input, an `Output` widget for displaying responses, and a `Button` widget to trigger interaction.
*   A function `on_button_click` was successfully defined and attached to the `ask_button`. This function handles user input by clearing previous output, retrieving the user's question, calling the `ask_question` function, displaying the chatbot's response (or an error/prompt), and then clearing the input area for the next query.
*   The complete interactive chatbot GUI, consisting of the text input area, the ask button, and the output display area, was successfully displayed in the IPython environment.

### Insights or Next Steps
*   The chatbot GUI is fully operational and ready for live testing to evaluate its responsiveness and the accuracy of the `ask_question` function.
*   Consider implementing enhanced error handling and user feedback mechanisms within the GUI to improve the user experience.
